In [1]:
import pandas as pd 
import numpy as np 

In [2]:
lof = pd.read_csv('../Datathon2020data/lof.csv')
kms = pd.read_csv('../Datathon2020data/kmeans.csv')
iso = pd.read_csv('../Datathon2020data/iso.csv')

In [3]:
lof['lof'].value_counts(normalize=True)

0    0.864979
1    0.135021
Name: lof, dtype: float64

In [4]:
kms['kmeans'].value_counts(normalize=True)

0    0.914923
1    0.085077
Name: kmeans, dtype: float64

In [5]:
iso['iso'].value_counts(normalize=True)

0    0.899998
1    0.100002
Name: iso, dtype: float64

### Merging all results

In [6]:
all_df = pd.concat([lof, kms[['kmeans']], iso[['iso']]], axis=1)

In [7]:
all_df['mutual_real'] = np.where(((all_df['lof']==1)&(all_df['kmeans']==1)&(all_df['iso']==1)), '1','0')
all_df['union_real'] = np.where((all_df['lof']==1)|(all_df['kmeans']==1)|(all_df['iso']==1), '1','0')

In [8]:
all_df['mutual_real'].value_counts(normalize=True)

0    0.994304
1    0.005696
Name: mutual_real, dtype: float64

In [9]:
all_df['union_real'].value_counts(normalize=True)

0    0.743303
1    0.256697
Name: union_real, dtype: float64

In [10]:
all_df.drop(['kmeans', 'iso'], axis=1, inplace=True)

### Merge back with non_fraud

In [11]:
claims = pd.read_csv('../cleaned_data/claims.csv')

#small fix from the last notebook 
claims['Age'] = claims['Age']//365
claims['Is_Dead'] = np.where(claims['Death_age'].isna(), False, True)
claims['DeductibleAmt'] = np.where(claims['DeductibleAmt'].isna(), 0, claims['DeductibleAmt'])

claims['InpatientAnnualDeductibleAmt'] = np.where(claims['InpatientAnnualDeductibleAmt']==0,0,1)

#Single grouping - amount less than 500
claims['OutpatientAnnualDeductibleAmt'] = np.where(claims['OutpatientAnnualDeductibleAmt']<=500,0,1)

claims['DeductibleAmt'] = np.where(claims['DeductibleAmt']==0, 0, 1)

#small fix
claims['Age'] = claims['Age']//365
claims['Is_Dead'] = np.where(claims['Death_age'].isna(), False, True)
claims['DeductibleAmt'] = np.where(claims['DeductibleAmt'].isna(), 0, claims['DeductibleAmt'])

claims['InpatientAnnualDeductibleAmt'] = np.where(claims['InpatientAnnualDeductibleAmt']==0,0,1)

#Single grouping - amount less than 500
claims['OutpatientAnnualDeductibleAmt'] = np.where(claims['OutpatientAnnualDeductibleAmt']<=500,0,1)

claims['DeductibleAmt'] = np.where(claims['DeductibleAmt']==0, 0, 1)

In [12]:
### TO-DO: split reimbursement amount into groups

In [13]:
non_fraud = claims[claims['Fraud']==0].reset_index(drop=True)

In [14]:
non_fraud.drop(['Death_age','Fraud'],axis=1, inplace=True)

In [15]:
non_fraud['union_real'] = '-1'
non_fraud['mutual_real'] = '-1'
non_fraud['lof'] = -1

In [16]:
df = pd.concat([non_fraud, all_df], axis=0).reset_index(drop=True)

In [74]:
#df.to_csv('../web_app/data/final.csv')

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558211 entries, 0 to 558210
Data columns (total 26 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   AmtReimbursed                     558211 non-null  int64 
 1   DeductibleAmt                     558211 non-null  int64 
 2   Is_inpatient                      558211 non-null  int64 
 3   Gender                            558211 non-null  int64 
 4   Race                              558211 non-null  int64 
 5   RenalDisease                      558211 non-null  int64 
 6   State                             558211 non-null  int64 
 7   County                            558211 non-null  int64 
 8   InpatientAnnualReimbursementAmt   558211 non-null  int64 
 9   InpatientAnnualDeductibleAmt      558211 non-null  int64 
 10  OutpatientAnnualReimbursementAmt  558211 non-null  int64 
 11  OutpatientAnnualDeductibleAmt     558211 non-null  int64 
 12  Nu

### Visualization

In [18]:
import plotly.express as px

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558211 entries, 0 to 558210
Data columns (total 26 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   AmtReimbursed                     558211 non-null  int64 
 1   DeductibleAmt                     558211 non-null  int64 
 2   Is_inpatient                      558211 non-null  int64 
 3   Gender                            558211 non-null  int64 
 4   Race                              558211 non-null  int64 
 5   RenalDisease                      558211 non-null  int64 
 6   State                             558211 non-null  int64 
 7   County                            558211 non-null  int64 
 8   InpatientAnnualReimbursementAmt   558211 non-null  int64 
 9   InpatientAnnualDeductibleAmt      558211 non-null  int64 
 10  OutpatientAnnualReimbursementAmt  558211 non-null  int64 
 11  OutpatientAnnualDeductibleAmt     558211 non-null  int64 
 12  Nu

In [20]:
pd.options.plotting.backend = "plotly"

In [27]:
df['union_real'].value_counts()

-1    345415
0     158172
1      54624
Name: union_real, dtype: int64

In [28]:
real = df[df['union_real']=='1']
potential = df[df['union_real']=='0']
non = df[df['union_real']=='-1']

In [32]:
real[['AmtReimbursed','ChronicDisease_Num',
    'Is_inpatient']].plot.scatter(
    x='ChronicDisease_Num', y='AmtReimbursed',
    size='ChronicDisease_Num', color='Is_inpatient')

In [40]:
df[df['AmtReimbursed']>8000].describe()

,AmtReimbursed,DeductibleAmt,Is_inpatient,Gender,Race,RenalDisease,State,County,InpatientAnnualReimbursementAmt,InpatientAnnualDeductibleAmt,OutpatientAnnualReimbursementAmt,OutpatientAnnualDeductibleAmt,NumOfClaimDays,Age,DiagnosisCode_Num,ProcedureCode_Num,ChronicDisease_Num,PhysiciansNum,lof
count,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000,16778.000000
mean,17825.334962,0.965431,0.989987,1.577006,1.248242,0.287341,25.634402,378.164263,25537.051496,0.992729,2346.477530,0.181845,7.212361,42.957265,8.267314,1.042019,5.501848,1.912862,-0.351115
std,12304.620865,0.182691,0.099566,0.494049,0.702140,0.452535,15.216511,266.078823,19436.590384,0.084965,5336.190149,0.385728,6.610144,37.337062,1.799580,0.788371,2.149900,0.560390,0.592747
min,8010.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000
25%,10000.000000,1.000000,1.000000,1.000000,1.000000,0.000000,11.000000,150.000000,12030.000000,1.000000,180.000000,0.000000,3.000000,0.000000,8.000000,1.000000,4.000000,2.000000,-1.000000
50%,13000.000000,1.000000,1.000000,2.000000,1.000000,0.000000,25.000000,341.000000,19000.000000,1.000000,750.000000,0.000000,5.000000,63.000000,9.000000,1.000000,5.000000,2.000000,0.000000
75%,20000.000000,1.000000,1.000000,2.000000,1.000000,1.000000,38.000000,570.000000,32000.000000,1.000000,2200.000000,0.000000,9.000000,76.000000,9.000000,1.000000,7.000000,2.000000,0.000000
max,125000.000000,1.000000,1.000000,2.000000,5.000000,1.000000,54.000000,999.000000,161470.000000,1.000000,102960.000000,1.000000,36.000000,100.000000,10.000000,5.000000,11.000000,3.000000,1.000000


In [45]:
amount= [(df['AmtReimbursed']<=300), 
         ((df['AmtReimbursed']>300)&(df['AmtReimbursed']<=1000)),
        ((df['AmtReimbursed']>1000)&(df['AmtReimbursed']<=8000)),
        ((df['AmtReimbursed']>8000)&(df['AmtReimbursed']<=20000)),
        (df['AmtReimbursed']>20000)]

In [46]:
level = ['under 300', '300-1k', '1k-8k', '8k-20k', '20k+']

In [47]:
df['Reimbursed_Level'] = np.select(amount, level)

In [48]:
df['Reimbursed_Level'].value_counts()

under 300    420392
300-1k        62943
1k-8k         58098
8k-20k        12810
20k+           3968
Name: Reimbursed_Level, dtype: int64

In [49]:
temp = df.groupby(['Reimbursed_Level', 'union_real']).count().reset_index()

In [53]:
reim = temp.pivot(index='Reimbursed_Level', columns='union_real',values='AmtReimbursed').reset_index()

In [55]:
reim.fillna(0, inplace=True)

In [64]:
reim['Reimbursed_Level'] = pd.Categorical(reim['Reimbursed_Level'],
                                         level)

In [67]:
reim.sort_values('Reimbursed_Level', inplace=True)

In [68]:
reim

union_real,Reimbursed_Level,-1,0,1,Fraud,Potential_Fraud,non_Fraud
4,under 300,266380.0,124391.0,29621.0,7.046043,29.589288,63.364669
2,300-1k,39787.0,19077.0,4079.0,6.480466,30.308374,63.211159
0,1k-8k,32321.0,13398.0,12379.0,21.307102,23.061035,55.631863
3,8k-20k,5360.0,1306.0,6144.0,47.962529,10.195160,41.842311
1,20k+,1567.0,0.0,2401.0,60.509073,0.000000,39.490927


In [57]:
reim['Fraud'] = 100*reim['1']/(reim['0']+reim['1']+reim['-1'])
reim['Potential_Fraud'] = 100*reim['0']/(reim['0']+reim['1']+reim['-1'])
reim['non_Fraud'] = 100*reim['-1']/(reim['0']+reim['1']+reim['-1'])

In [70]:
fig1 = reim.plot.bar(x='Reimbursed_Level', y=['Fraud',
                                            'Potential_Fraud',
                                            'non_Fraud'])

In [71]:
fig1.show()